# NC Teacher Attrition Data 2014-2015
* This program downloads all original teacher attrition rate by reason datasets from http://www.ncpublicschools.org/docs/district-humanresources/surveys/leaving/ and saves them as .csv files. These data files are used to create all the flattened and machine learning datasets
    * The data is allocated on the appendix of the yearly NC state teaching profession report PDF file page 23 to page 27. 
    * Each page has one table, the program loops though each page and extract the data to a list
    * Some LEA names are too long which will expand to multiple rows, I fixed them to original name as well.
    * The final result will be saved into an csv file into RawDataset folder.

In [1]:
import PyPDF2
import requests 
from tabula import read_pdf
import tabula
import pandas as pd
import urllib
import warnings
warnings.simplefilter('ignore')
warnings.simplefilter('ignore', DeprecationWarning)
pd.set_option('mode.chained_assignment', None)


In [2]:
PDFfilename = "2014-15turnoverreport.pdf"

url = 'http://www.ncpublicschools.org/docs/district-humanresources/surveys/leaving/' + PDFfilename
r = requests.get(url)
# downlaod the file
with open(PDFfilename, 'wb') as f:  
    f.write(r.content)

pfr = PyPDF2.PdfFileReader(open(PDFfilename,"rb"))
NewPDFfilename = "TeacherAttrition_ByReasonsCategories"

files = [] 
for i in range(23,27):
    pg = pfr.getPage(i-1)
    writer = PyPDF2.PdfFileWriter() 
    writer.addPage(pg)
    PDF = NewPDFfilename + "_"+str(i) +".pdf"
    files.append(PDF)
    with open(PDF,"wb") as outputStream:
        writer.write(outputStream)


In [3]:
# read the pages from each individual pdf file.
df=pd.DataFrame([])
df_combine=pd.DataFrame([])

for file in files:
            df = tabula.read_pdf(file,multiple_tables=False)
            df.columns = ['LEACode', 'Lea_Name', 'Attrition_Reason_TotalTeachers', 'Attrition_Reason_TeachersLeaving'
                      ,'Attrition_Reason_RemainedinEducation','Attrition_Reason_RemainedinEducationPerct', 'Attrition_Reason_PersonalReasons'
                            ,'Attrition_Reason_PersonalPerct','Attrition_Reason_InitiatedbyLEA','Attrition_Reason_InitiatedPerct'
                      ,'Attrition_Reason_BeyondControlofLEA','Attrition_Reason_BeyondControlPerct'
                           ,'Attrition_Reason_OtherReasons','Attrition_Reason_OtherReasonsPerct']
            #If you want to change the table by editing the columns you can do that here.

            df_combine=pd.concat([df_combine,df],axis=0,sort=False,ignore_index=True) #again you can choose between merge or concat as per your need
            df_combine.columns = ['LEACode', 'Lea_Name', 'Attrition_Reason_TotalTeachers', 'Attrition_Reason_TeachersLeaving'
                      ,'Attrition_Reason_RemainedinEducation','Attrition_Reason_RemainedinEducationPerct', 'Attrition_Reason_PersonalReasons'
                            ,'Attrition_Reason_PersonalPerct','Attrition_Reason_InitiatedbyLEA','Attrition_Reason_InitiatedPerct'
                      ,'Attrition_Reason_BeyondControlofLEA','Attrition_Reason_BeyondControlPerct'
                           ,'Attrition_Reason_OtherReasons','Attrition_Reason_OtherReasonsPerct']

exclude = ['LEA','Code']

df = df_combine[~df_combine['LEACode'].isin(exclude) & ~ df_combine['LEACode'].isna()] # remove LEA code NaN as well as some 
# fix the name cross multiple line
df.loc[df['LEACode']=='700','Lea_Name']='Elizabeth City-Pasquotank Public Schools'
df.loc[df['LEACode']=='94Z','Lea_Name']='Northeast Regional School - Biotech/Agri'
df.loc[df['LEACode']=='340','Lea_Name']='Winston Salem/Forsyth County Schools'

df.loc[df['LEACode']=='681','Lea_Name']='Chapel Hill-Carrboro Schools'
df.loc[df['LEACode']=='330','Lea_Name']='Edgecombe County Public School'
df.loc[df['LEACode']=='340','Lea_Name']='Forsyth County Schools'
df.loc[df['LEACode']=='500','Lea_Name']='Jackson County Schools'
df.loc[df['LEACode']=='491','Lea_Name']='Mooresville City Schools'
df.loc[df['LEACode']=='700','Lea_Name']='Pasquotank County Schools'
df.loc[df['LEACode']=='760','Lea_Name']='Randolph County Schools'
df.loc[df['LEACode']=='780','Lea_Name']='Robeson County Schools'

In [4]:

for col in df.columns:
    if(col.find("TotalTeachers")>0):
        #print(col)
        df[col] = df[col].str.replace(",","")
    if(col.find("Perct")>0):
        df[col] = df[col].str.replace("%","")

In [5]:
df.head(20)

,LEACode,Lea_Name,Attrition_Reason_TotalTeachers,Attrition_Reason_TeachersLeaving,Attrition_Reason_RemainedinEducation,Attrition_Reason_RemainedinEducationPerct,Attrition_Reason_PersonalReasons,Attrition_Reason_PersonalPerct,Attrition_Reason_InitiatedbyLEA,Attrition_Reason_InitiatedPerct,Attrition_Reason_BeyondControlofLEA,Attrition_Reason_BeyondControlPerct,Attrition_Reason_OtherReasons,Attrition_Reason_OtherReasonsPerct
3,010,Alamance-Burlington Schools,1507,231,108,46.75,38,16.45,58,25.11,10,4.33,17,7.36
4,020,Alexander County Schools,338,29,16,55.17,4,13.79,6,20.69,2,6.90,1,3.45
5,030,Alleghany County Schools,121,14,3,21.43,3,21.43,7,50.00,1,7.14,0,0.00
6,040,Anson County Schools,232,39,16,41.03,6,15.38,16,41.03,0,0.00,1,2.56
7,050,Ashe County Schools,237,22,8,36.36,8,36.36,3,13.64,2,9.09,1,4.55
8,761,Asheboro City Schools,356,49,21,42.86,4,8.16,24,48.98,0,0.00,0,0.00
9,111,Asheville City Schools,327,56,17,30.36,7,12.50,24,42.86,2,3.57,6,10.71
10,060,Avery County Schools,164,10,7,70.00,1,10.00,2,20.00,0,0.00,0,0.00
11,070,Beaufort County Schools,500,66,27,40.91,9,13.64,28,42.42,0,0.00,2,3.03
12,080,Bertie County Schools,185,57,28,49.12,9,15.79,15,26.32,2,3.51,3,5.26


In [6]:
df[df['LEACode']=='180']
len(df)

112

In [7]:
# save to csv file
csv = "../SchoolTeacherAttritionByReason2014.csv"

#df_sorted=df.sort_values(by=1, ascending=True, axis=0)
df_sorted = df

df_sorted.to_csv(csv,index=False)